In [112]:
from google.colab import drive
from google.colab import auth
import gspread
from google.auth import default
from keras import models
from keras.models import load_model
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [113]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [114]:
worksheet = gc.open('Goldpricedatabase').sheet1
rows = worksheet.get_all_values()
df = pd.DataFrame(rows)

In [115]:
df_values = df.iloc[:, [0, 4]]

In [116]:
df_values.columns = ['Date', 'Value']

In [117]:
df_values.to_csv('LatestValue.csv', index=False)

In [118]:
df = pd.read_csv("LatestValue.csv")

In [119]:
df = df.dropna()

In [120]:
model = load_model('/content/gdrive/MyDrive/Bestmodel')

In [121]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df['Value'].values.reshape(-1, 1))

lookback = 200
X_forecast = scaled_data[-lookback:] 

horizon = 7

forecasted_values = []

for i in range(horizon):
    X_forecast_reshaped = X_forecast.reshape(1, -1, 1)
    prediction = model.predict(X_forecast_reshaped)
    forecasted_values.append(prediction[0, 0])
    X_forecast = np.append(X_forecast, prediction, axis=0)
    X_forecast = X_forecast[1:].reshape(lookback, 1)

forecasted_values = np.array(forecasted_values).reshape(-1, 1)
forecasted_values = scaler.inverse_transform(forecasted_values)

last_date = pd.to_datetime(df['Date'].iloc[-1])
forecasted_dates = pd.date_range(start=last_date + pd.DateOffset(days=1), periods=horizon).strftime('%Y-%m-%d').tolist()

forecasted_df = pd.DataFrame({'Date': forecasted_dates, 'Value': forecasted_values.flatten(), 'Predictions': forecasted_values.flatten()})

df = pd.concat([df, forecasted_df], ignore_index=True)

# Print the forecasted values
print("Forecasted Values:")
print(df[df['Date'].isin(forecasted_dates)][['Date', 'Predictions']])



1/1 [==============================] - 0s 91ms/step
Forecasted Values:
           Date  Predictions
208  2023-05-28  1952.766602
209  2023-05-29  1949.733154
210  2023-05-30  1947.358398
211  2023-05-31  1945.480469
212  2023-06-01  1943.914429
213  2023-06-02  1942.536499
214  2023-06-03  1941.268066


In [122]:
forecasted_df.to_csv('Onestepdata.csv', index=False)

In [123]:
ws = gc.open('OneStep').sheet1

In [124]:
onestepf = forecasted_df.values.tolist()

ws.update('A1', onestepf)

print("DataFrame written successfully to the Google Sheet!")

DataFrame written successfully to the Google Sheet!


In [125]:
df = pd.read_csv("LatestValue.csv")
pd.read_csv('LatestValue.csv')
df = df.dropna()
print(df)

           Date        Value
1    11/01/2022  1647.124583
2    11/02/2022  1633.907386
3    11/03/2022  1629.706667
4    11/04/2022  1681.325000
5    11/05/2022  1681.325000
..          ...          ...
204   5/23/2023  1962.300000
205   5/24/2023  1973.264740
206   5/25/2023  1958.127253
207   5/26/2023  1950.786884
208   5/27/2023  1946.997058

[208 rows x 2 columns]


In [126]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df['Value'].values.reshape(-1, 1))

lookback = 200
X_forecast = scaled_data[-lookback:] 

horizon = 30

forecasted_values = []

for i in range(horizon):
    X_forecast_reshaped = X_forecast.reshape(1, -1, 1)
    prediction = model.predict(X_forecast_reshaped)
    forecasted_values.append(prediction[0, 0])
    X_forecast = np.append(X_forecast, prediction, axis=0)
    X_forecast = X_forecast[1:].reshape(lookback, 1)

forecasted_values = np.array(forecasted_values).reshape(-1, 1)
forecasted_values = scaler.inverse_transform(forecasted_values)

last_date = pd.to_datetime(df['Date'].iloc[-1])
forecasted_dates = pd.date_range(start=last_date + pd.DateOffset(days=1), periods=horizon).strftime('%Y-%m-%d').tolist()

forecasted_df = pd.DataFrame({'Date': forecasted_dates, 'Value': forecasted_values.flatten(), 'Predictions': forecasted_values.flatten()})

df = pd.concat([df, forecasted_df], ignore_index=True)

# Print the forecasted values
print("Forecasted Values for 1 Month Ahead:")
print(df[df['Date'].isin(forecasted_dates)][['Date', 'Predictions']])



1/1 [==============================] - 0s 21ms/step
Forecasted Values for 1 Month Ahead:
           Date  Predictions
208  2023-05-28  1952.766602
209  2023-05-29  1949.733154
210  2023-05-30  1947.358398
211  2023-05-31  1945.480469
212  2023-06-01  1943.914429
213  2023-06-02  1942.536499
214  2023-06-03  1941.268066
215  2023-06-04  1940.062744
216  2023-06-05  1938.892822
217  2023-06-06  1937.744629
218  2023-06-07  1936.610840
219  2023-06-08  1935.488892
220  2023-06-09  1934.377563
221  2023-06-10  1933.277222
222  2023-06-11  1932.188110
223  2023-06-12  1931.110718
224  2023-06-13  1930.045166
225  2023-06-14  1928.990967
226  2023-06-15  1927.948486
227  2023-06-16  1926.917236
228  2023-06-17  1925.896729
229  2023-06-18  1924.886353
230  2023-06-19  1923.886108
231  2023-06-20  1922.895264
232  2023-06-21  1921.913086
233  2023-06-22  1920.939697
234  2023-06-23  1919.974487
235  2023-06-24  1919.016846
236  2023-06-25  1918.066772
237  2023-06-26  1917.123901


In [127]:
forecasted_df.to_csv('horizondata.csv', index=False)

In [128]:
ws = gc.open('Horizon').sheet1

In [129]:
horizonf = forecasted_df.values.tolist()
ws.update('A1', horizonf)
print("DataFrame written successfully to the Google Sheet!")

DataFrame written successfully to the Google Sheet!
